In [27]:
import bw2io as bi
import bw2data as bd
import bw2calc as bc
from bw_temporalis import easy_timedelta_distribution, TemporalisLCA
import bw_processing as bwp
from utilities import *

# Set project and databases

In [28]:
bd.projects.set_current('tictac')

In [34]:
bd.databases

Databases dictionary with 7 object(s):
	bio
	wind-example
	wind-example-2020
	wind-example-2030
	wind-example-2035
	wind-example-2040
	wind-example-2050

In [33]:
#create a dummy biosphere database
bd.Database("bio").write({
        ("bio", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
            "unit": "kilogram",
        },
        ("bio", "coal"): {
            "type": "emission",
            "name": "coal",
            "unit": "kilogram",
        },
})

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 


In [35]:
def generate_simple_database(year, share_of_wind_in_electricity_mix, generated_electricity_over_lifetime=1000):
    """
    Write in the project a simple database for our testing for a given year.

    :param year: Year of the database.
    :param share_of_wind_in_electricity_mix: Share of wind electricity in the dummy electricity mix.
    :param generated_electricity_over_lifetime: Generated electricity by the wind turbine over its lifetime.
    :return: Database with the name 'wind-electricity-year'.
    """
    database_name = 'wind-example-'+str(year)
    
    if database_name in list(bd.databases):
        del bd.databases[database_name]
        
    bd.Database(database_name).write({
        (database_name, 'electricity-mix'): {
            'name': 'Electricity mix',
            'unit': 'kilowatt hour',
            'exchanges': [
                {
                    'input': (database_name, 'electricity-production-wind'),
                    'amount': share_of_wind_in_electricity_mix,
                    'type': 'technosphere',
                },
                {
                    'input': (database_name, 'electricity-production-coal'),
                    'amount': 1-share_of_wind_in_electricity_mix,
                    'type': 'technosphere',
                },
            ]
        },
        (database_name, 'electricity-production-coal'): {
            'name': 'Electricity production, coal',
            'unit': 'kilowatt hour',
            'exchanges': [
                {
                    'input': ("bio", 'coal'),
                    'amount': 1,
                    'type': 'biosphere',
                },
                {
                    'input': ("bio", 'CO2'),
                    'amount': 1,
                    'type': 'biosphere',
                },
            ]
        },
        (database_name, 'electricity-production-wind'): {
            'name': 'Electricity production, wind',
            'unit': 'kilowatt hour',
            'exchanges': [
                {
                    'input': (database_name, 'electricity-production-wind'),
                    'amount': 1,
                    'type': 'production',
                },
                {
                    'input': (database_name, 'wind-turbine-construction'),
                    'amount': 1/generated_electricity_over_lifetime,
                    'type': 'technosphere',
                    'temporal_distribution': easy_timedelta_distribution(
                        start=-5,
                        end=0,
                        resolution="Y",  # M for months, Y for years, etc.
                        steps=6,
                    ),
                },
                # maybe add maintenance, oil changes, ...
            ]
        },
        (database_name, 'wind-turbine-construction'): {
            'name': 'Wind turbine construction',
            'unit': 'unit',
            'exchanges': [
                {
                    'input': (database_name, 'wind-turbine-construction'),
                    'amount': 1,
                    'type': 'production'
                },
                {
                    'input': (database_name, 'electricity-mix'),
                    'amount': 1,
                    'type': 'technosphere',
                },
                # {
                #     'input': (database_name, 'eol-wind'),
                #     'amount': 1,
                #     'type': 'technosphere',
                #     'temporal_distribution': easy_timedelta_distribution(
                #         start=0,
                #         end=20,
                #         resolution="Y",  # M for months, Y for years, etc.
                #         steps=10,
                #     ),
                # },
                # aggregate the rest to direct co2 emissions
                {
                    'input': ("bio", 'CO2'),
                    'amount': 1,
                    'type': 'biosphere',
                }
            ]
        },
        (database_name, 'eol-wind'): {
            'name': "End-of-life, wind turbine",
            'unit': 'unit',
            'exchanges': [
                {
                    'input': (database_name, 'eol-wind'),
                    'amount': 1,
                    'type': 'production',
                },
                # aggregate the rest to direct co2 emissions
                {
                    'input': ("bio", 'CO2'),
                    'amount': 1,
                    'type': 'biosphere',
                }
            ]
        },
    })

In [36]:
generate_simple_database(2020, 0.3)
generate_simple_database(2030, 0.5)
generate_simple_database(2040, 0.6)
generate_simple_database(2050, 0.7)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

Vacuuming database 


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1249.64it/s]

Vacuuming database 


# Timeline generation calculation

### Impact assessment methods

In [37]:
bd.Method(("GWP", "wind-example-2020")).write([
    (('bio', "CO2"), 1),
])

### Static LCA calculation

In [38]:
fu_2020 = {('wind-example-2020', 'electricity-production-wind'): 1}

In [39]:
lca = bc.LCA(fu_2020, ("GWP", "wind-example-2020"))
lca.lci()
lca.lcia()
lca.score

0.0017005102219358214

### Dynamic LCA with temporalis

In [40]:
tlca = TemporalisLCA(lca)

Starting graph traversal
Calculation count: 3


In [41]:
tl = tlca.build_timeline(node_timeline=True)
tl.build_dataframe()
tl_df = tl.add_metadata_to_dataframe(database_labels=["wind-example-2020"], fields=['name', 'unit'])

C:\Users\MULLERA\Anaconda3\envs\tictac\lib\site-packages\bw_temporalis\lca.py:135: UserWarning: This functionality is experimental, and will change.
You have been warned.
  warnings.warn(


# Patch database with prospective databases

### Information on existing databases

In [51]:
database_date_dict = {
    2020: 'wind-example-2020',
    2030: 'wind-example-2030',
    2040: 'wind-example-2040',
    2050: 'wind-example-2050',
}   

In [52]:
dates_list = [datetime.strptime(str(x), "%Y") for x in database_date_dict.keys()]

### Example of a patch

In [53]:
exchange_output = ('wind-example-2050', 'electricity-production-wind')
old_exchange_input = ('wind-example-2050', 'wind-turbine-construction')
new_exchange_input = ('wind-example-2030', 'wind-turbine-construction')
new_exchange_input_amount = 0.001

In [54]:
dp = safety_razor(
    consumer=exchange_output,
    previous_producer=old_exchange_input, 
    new_producer=new_exchange_input, 
)

In [55]:
demand = {('wind-example-2050', 'electricity-production-wind'): 1, ('wind-example-2030', 'electricity-production-wind'): 1}
ia = ("GWP", "wind-example-2020")
fu, data_objs, remapping = bd.prepare_lca_inputs(demand=demand, method=ia)

In [56]:
lca_patched = bc.LCA(fu, data_objs=data_objs + [dp], remapping_dicts=remapping)
lca_patched.lci()
lca_patched.lcia()
lca_patched.score

0.0030015008217283224

In [57]:
tlca_patched = TemporalisLCA(lca_patched)

Starting graph traversal
Calculation count: 7


In [58]:
tl_patched = tlca_patched.build_timeline(node_timeline=True)
tl_patched.build_dataframe()
tl_patched_df = tl_patched.add_metadata_to_dataframe(database_labels=["wind-example-2035"], fields=['name', 'unit'])

C:\Users\MULLERA\Anaconda3\envs\tictac\lib\site-packages\bw_temporalis\lca.py:135: UserWarning: This functionality is experimental, and will change.
You have been warned.
  warnings.warn(


### Generating interpolation weights

In [59]:
tl_df = add_column_interpolation_weights_on_timeline(tl_df, dates_list)
tl_df

C:\Users\MULLERA\OneDrive - VITO\Documents\12_teaching\2023_Autumn_school_time_in_LCA\group_work\tictac_lca\Mini-Wind-Example\utilities.py:184: Warning: Date outside the range of dates covered by the databases.
  warnings.warn("Date outside the range of dates covered by the databases.", category=Warning)


,date,amount,flow,activity,activity_name,activity_unit,flow_name,flow_unit,interpolation_weights
0,2018-10-12 13:00:36,0.000167,-1,258,Wind turbine construction,unit,NaN,NaN,{2020: 1}
1,2018-10-12 13:00:36,0.000167,-1,255,Electricity mix,kilowatt hour,NaN,NaN,{2020: 1}
2,2018-10-12 13:00:36,0.000117,-1,256,"Electricity production, coal",kilowatt hour,NaN,NaN,{2020: 1}
3,2019-10-12 18:49:48,0.000167,-1,258,Wind turbine construction,unit,NaN,NaN,{2020: 1}
4,2019-10-12 18:49:48,0.000167,-1,255,Electricity mix,kilowatt hour,NaN,NaN,{2020: 1}
5,2019-10-12 18:49:48,0.000117,-1,256,"Electricity production, coal",kilowatt hour,NaN,NaN,{2020: 1}
6,2020-10-12 00:39:00,0.000167,-1,255,Electricity mix,kilowatt hour,NaN,NaN,"{2020: 0.07802548133953828, 2030: 0.9219745186..."
7,2020-10-12 00:39:00,0.000167,-1,258,Wind turbine construction,unit,NaN,NaN,"{2020: 0.07802548133953828, 2030: 0.9219745186..."
8,2020-10-12 00:39:00,0.000117,-1,256,"Electricity production, coal",kilowatt hour,NaN,NaN,"{2020: 0.07802548133953828, 2030: 0.9219745186..."
9,2021-10-12 06:28:12,0.000167,-1,258,Wind turbine construction,unit,NaN,NaN,"{2020: 0.17800974085226753, 2030: 0.8219902591..."


# Testing automated patching

In [60]:
# Creating a mock timeline

import pandas as pd
import numpy as np

# Number of rows in the DataFrame
n = 10

# Generate random dates
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2050-01-01')
date_range = (end_date - start_date).days
dates = [start_date + pd.to_timedelta(np.random.randint(0, date_range), unit='D') for _ in range(n)]

# Generate random amounts, let's say between 10 and 1000
amounts = np.random.randint(10, 1000, size=n)

# Generate random producer and consumer integers, let's say between 1 and 10
producers = np.random.randint(213, 230, size=n)
consumers = np.random.randint(213, 230, size=n)

# Create DataFrame
df = pd.DataFrame({
    'date': dates,
    'amount': amounts,
    'producer': producers,
    'consumer': consumers
}).sort_values(by='date')

In [61]:
df = add_column_interpolation_weights_on_timeline(df, dates_list)

In [62]:
df

,date,amount,producer,consumer,interpolation_weights
8,2023-09-17,774,228,222,"{2020: 0.37092800437996165, 2030: 0.6290719956..."
7,2024-02-09,144,224,219,"{2020: 0.4106214070626882, 2030: 0.58937859293..."
6,2024-03-01,237,225,216,"{2020: 0.41637010676156583, 2030: 0.5836298932..."
4,2027-04-28,898,215,215,"{2020: 0.7320010949904189, 2030: 0.26799890500..."
1,2034-04-23,660,220,214,"{2030: 0.4307228915662651, 2040: 0.56927710843..."
5,2034-11-07,171,216,223,"{2030: 0.48493975903614456, 2040: 0.5150602409..."
9,2036-03-02,229,228,215,"{2030: 0.6166484118291348, 2040: 0.38335158817..."
0,2037-12-29,681,219,224,"{2030: 0.7992880613362541, 2040: 0.20071193866..."
2,2041-09-08,70,217,217,"{2040: 0.16862852450041063, 2050: 0.8313714754..."
3,2045-12-04,208,226,227,"{2040: 0.5923898165891048, 2050: 0.40761018341..."


In [63]:
bd.get_activity(('wind-example-2020', 'electricity-mix')).id

255

In [64]:
def extract_edge_from_row(row):
    new_edges = []
    consumer = bd.get_node(id=row['consumer'])
    previous_producer = bd.get_node(id=row['producer'])
    for date, share in row['interpolation_weights'].items():
        database = database_date_dict[date]
        new_amount = row['amount'] * share
        new_producer = bd.get_activity(key=(database, previous_producer['code']))
        new_edges.append((consumer, previous_producer, new_producer, new_amount))
    
    return consumer

extract_edge_from_row(df.iloc[2])

# dp = safety_razor(
#     consumer=exchange_output,
#     previous_producer=old_exchange_input, 
#     new_producer=new_exchange_input, 
# )

UnknownObject: 

In [65]:
df.iloc[0]['interpolation_weights']

{2020: 0.37092800437996165, 2030: 0.6290719956200383}